# Install Package yang diperlukan

In [ ]:
!pip install jcopdl

# Import Package yang diperlukan

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from jcopdl.layers import linear_block
from torch import nn

import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Datasets dan DataLoader

In [ ]:
# Download Datasets
!unrar x "/content/drive/My Drive/datasets/Intel_Image_Classification.rar" "/content/"

In [9]:
bs = 256

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalize supaya menjadi -1 sampai 1, supaya lebih stabil
])

train_set = datasets.ImageFolder("/content/Intel_Image_Classification/train/", transform=data_transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=8)

In [11]:
image, label = next(iter(trainloader))
image.shape

torch.Size([256, 3, 150, 150])

# Set Config

In [12]:
# set config
config = set_config({
    "z_dim" : 512,
    "bs" :256
})

# Training Preparation

In [13]:
from model_gan import Discriminator, Generator

In [14]:
D_model = Discriminator().to(device)
G_model = Generator(config.z_dim).to(device)

criterion = nn.BCELoss()

d_optimizer = optim.Adam(D_model.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G_model.parameters(), lr=0.0002)

# Proses Training

In [15]:
import os
from torchvision.utils import save_image

os.makedirs("output/GAN/", exist_ok=True)
os.makedirs("model/GAN/", exist_ok=True)

In [16]:
num_epochs = 200

for epoch in range(num_epochs):
    D_model.train()
    G_model.train()
    
    for real_image, _ in trainloader:
        jumlah_data = real_image.shape[0]
        
        # Handle Real image dan Fake image
        real_image = real_image.to(device)
        fake_image = G_model.generate(jumlah_data, device)
        
        # Handle Real label dan Fake label
        real_label = torch.ones((jumlah_data, 3), device=device)
        fake_label = torch.zeros((jumlah_data, 3), device=device)
        
        # Training Discriminator
        d_optimizer.zero_grad()
        ## 1. Real image -> Discriminator -> label real
        output = D_model(real_image)
        d_real_loss = criterion(output, real_label)
        
        ## 2. Fake image -> Discriminator -> label fake
        output = D_model(fake_image.detach())
        d_fake_loss = criterion(output, fake_label)
        
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()
        
        # Training Generator
        g_optimizer.zero_grad()
        ## Fake image -> Discriminator -> label real -> jangan di detach()
        output = D_model(fake_image)
        g_loss = criterion(output, real_label)
        g_loss.backward()
        g_optimizer.step()
        
        
    if epoch % 5 == 0:
        print(f"Epoch: {epoch:5} | D_loss: {d_loss/2:.5f} | G_loss: {g_loss/2:.5f}")
        
    if epoch % 20 == 0:
        G.eval()
        epoch = str(epoch).zfill(4)
        fake_image = G_model.generate(64, device)
        save_image(fake_image.view(-1, 3, 150, 150), f"output/GAN/{epoch}.jpg", nrow=8, normalize=True)
        
        #save model
        torch.save(D, "model/GAN/discriminator.pth")
        torch.save(G, "model/GAN/generator.pth")

RuntimeError: ignored